In [5]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imutils
from imutils import perspective
import math
import time

In [6]:
resW = 1920
resH = 1080

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resW)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resH)
#cap.set(cv2.CAP_PROP_FPS, 20)

font = cv2.FONT_HERSHEY_TRIPLEX
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

sizeOut = 512
colorFPS = (255, 255, 0)
colorRect = (0, 255, 0)

#flags
detected = 0
capture = 0
save = 0

#parameters
paraGaussian = 5
paraCannyMin = 75
paraCannyMax = 200

textStringA = ""
textStringB = ""

In [7]:
while(True):
    tick1 = cv2.getTickCount()
    (grabbed, img) = cap.read()
    
    if not grabbed:
        break
    
    imgOrig = img.copy()
    detected = 0
    
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #imgCla = clahe.apply(imgGray)
    imgGray = cv2.GaussianBlur(imgGray, (paraGaussian, paraGaussian), 0)
    imgEdged = cv2.Canny(imgGray, paraCannyMin, paraCannyMax)
    (_, contours, _) = cv2.findContours(imgEdged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:5]

    for c in contours:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        if len(approx) == 4:
            screenCnt = approx
            detected = 1
            break

    if detected == 1:
        imgCont = cv2.drawContours(img, [screenCnt], -1, colorRect, 2)
        imgWarp = perspective.four_point_transform(imgOrig, screenCnt.reshape(4, 2))
            
        tick2 = cv2.getTickCount()
        tick = math.floor(((tick2 - tick1) * 1000) / cv2.getTickFrequency())
        imgCont = cv2.copyMakeBorder(imgCont, 0, 80, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        textStringA = str(tick) + " ms " + str(paraCannyMin) + " " + str(paraCannyMax)
        cv2.putText(imgCont, textStringA, (10, resH+30), font, 1, colorFPS, 2, cv2.LINE_AA)
        cv2.putText(imgCont, textStringB, (10, resH+70), font, 1, colorFPS, 2, cv2.LINE_AA)
        
        imgEdged = imutils.resize(imgEdged, height = sizeOut)
        cv2.imshow("Edged", imgEdged)
        imgCont = imutils.resize(imgCont, height = sizeOut)
        cv2.imshow("Cont", imgCont)
        imgWarp = imutils.resize(imgWarp, height = sizeOut)
        cv2.imshow("Warp", imgWarp)
        if capture == 1:
            save = 1
            imgWarpCap = imgWarp.copy()
            cv2.imshow("imgWarpCap", imgWarpCap)
        
    
    capture = 0
    getKey = cv2.waitKey(20) & 0xFF
    if getKey == ord('c'):
        capture = 1
    elif getKey == ord('z'):
        paraCannyMin += 25
        paraCannyMax += 25
    elif getKey == ord('x'):
        paraCannyMin -= 25
        paraCannyMax -= 25
    elif getKey == ord('v'):
        if save == 1:
            fileName = "../imgCapture/" + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.png'
            cv2.imwrite(fileName, imgWarpCap, [int(cv2.IMWRITE_PNG_COMPRESSION), 9])
            textStringB = "file saved: " + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.png'
            save = 0
    elif getKey == ord('q'):
        break

In [8]:
cap.release()
cv2.destroyAllWindows()